In [13]:
using Oscar
using Polymake

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.5.1 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2021 by The Oscar Development Team


In [14]:
tv = fulton.NormalToricVariety(INPUT_RAYS=[0 1; 1 0; 0 -1; -2 1]; INPUT_CONES=[[0,1],[1,2],[2,3],[0,3]])

type: NormalToricVariety

INPUT_CONES
	{0 1}
	{1 2}
	{2 3}
	{0 3}

INPUT_RAYS
	0 1
	1 0
	0 -1
	-2 1

In [43]:
l = tv.MAXIMAL_CONES
println(l)
println(l[1,:])
println(Array(l))

pm::IncidenceMatrix<pm::NonSymmetric>
{0 1}
{1 2}
{2 3}
{0 3}

Bool[1, 1, 0, 0]
CxxWrap.CxxWrapCore.CxxBool[true true false false; false true true false; false false true true; true false false true]


In [31]:
function slicematrix(A::AbstractMatrix)
    return [A[i, :] for i in 1:size(A,1)]
end

slicematrix (generic function with 1 method)

In [32]:
function rowMinors(A::AbstractMatrix,S::Set)
    outList=[]
    slices=slicematrix(A)
    for i in 1:size(A,1)
        if i in S
            append!(outList,slices[i])
        end
    end
    return transpose(hcat(outList...))
end

rowMinors (generic function with 1 method)

In [36]:
function coneListFormat(coneList, X)
    memberList=[]
    denseList=slicematrix(Array(Polymake.common.dense(coneList)))
    for row in denseList
        members=[]
        for i in 1:size(X,1)
            if row[i]==1
                append!(members,i-1)
            end
        end
        append!(memberList,members)
    end
    return memberList
end

coneListFormat (generic function with 2 methods)

In [37]:
println(coneListFormat(l, tv.RAYS))

Any[0, 1, 1, 2, 2, 3, 0, 3]


In [20]:
tv.RAYS

pm::Matrix<pm::Rational>
0 1
1 0
0 -1
-1 1/2


In [21]:
size(tv.RAYS, 1)

4

In [24]:
function makeSmooth(X)
    Y  = copy(X)
    while(true)
        coneList = Y.MAXIMAL_CONES
        rayMatrix = Y.RAYS
        k = 1
        for cone in coneList
            coneSet=coneListFormat(cone)
            S=size(coneSet)[1]
            coneRays=rowMinors(coneList,coneSet)
            R=rank(coneRays)
            smoothCheck=Polymake.fan.check_fan_objects(Polymake.polytope.Cone(RAYS=coneRays)).SMOOTH_FAN
            if R != S  !smoothCheck
                break
            else
                k+=1
            end
        end
        if k == 0
            break
        end
        sigma=coneListFormat(coneList[k])
        for subset in powerset(sigma,min=2,max=length(sigma))
            S=size(subset)[1]
            subsetRays=rowMinors(coneList,subset)
            R=rank(subsetRays)
            smoothCheck=Polymake.fan.check_fan_objects(Polymake.polytope.Cone(Rays=subsetRays)).SMOOTH_FAN
            if R != S  !smoothCheck
                tau=subset
                break
            end
        end
    end
end


makeSmooth (generic function with 1 method)

In [25]:
makeSmooth(tv)

LoadError: [91mno matching overloaded instance of Polymake::common::dense(Integer) at -e line 0.[39m
[91m	eval {...} called at -e line 0[39m
